<a href="https://colab.research.google.com/github/GitYCC/bert-minimal-tutorial/blob/master/toxic_comment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Toxic Comment Classification

In [1]:
!git clone https://github.com/GitYCC/bert-minimal-tutorial.git

Cloning into 'bert-minimal-tutorial'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 34 (delta 13), reused 22 (delta 4), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [2]:
%cd bert-minimal-tutorial

/content/bert-minimal-tutorial


In [3]:
!pip install -q -r requirements.txt

     |████████████████████████████████| 829kB 10.5MB/s 
     |████████████████████████████████| 1.3MB 50.4MB/s 
     |████████████████████████████████| 512kB 50.9MB/s 
     |████████████████████████████████| 727kB 49.5MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 890kB 52.3MB/s 
     |████████████████████████████████| 1.1MB 49.4MB/s 
     |████████████████████████████████| 2.9MB 51.9MB/s 
     |████████████████████████████████| 1.3MB 53.3MB/s 
     |████████████████████████████████| 133kB 52.1MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [4]:
import os

import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer, BertModel, BertPreTrainedModel
from tqdm.notebook import tqdm
from sklearn import metrics

from utils import RunningAverage

MODEL_NAME = 'bert-base-uncased'
SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

## Dataloader

In [5]:
df = pd.read_csv('data/toxic_comment_classification.csv')
df = df.sample(frac=1).reset_index(drop=True)  # shuffle

In [6]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,696535ea776b2640,"""Please explain the terms """"tinfoil hattery"""" ...",0,0,0,0,0,0
1,b09ca6bd86c0ccd3,REDIRECT Talk:Rough-bellied day gecko,0,0,0,0,0,0
2,a37fea589a51f37d,"No problem, Ylee; happens to me all the time. ...",0,0,0,0,0,0
3,0569a07e22b86f53,"Siafu, I am getting access denied for GPS PPS....",0,0,0,0,0,0
4,c6c204a4b121ab89,"Yeah, please delete the semi-protected option....",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,a0bbb93ce776d766,"""\n\n Tagging article Economy of Bihar \n\nIt ...",0,0,0,0,0,0
159567,bb9a3bd44f946e6f,"""\n\nFirst off, T.N.E. has always had a stron...",0,0,0,0,0,0
159568,6c395910bbf66153,Thanks for creating Khamenei's fatwa against n...,0,0,0,0,0,0
159569,3865f8ff94c06e88,"""\n\n Please stop defending pedophilia on MY t...",0,0,0,0,0,0


In [7]:
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [8]:
class MultiLabelDataset(Dataset):
    def __init__(self, tokenizer, df, max_len=512, for_train=True):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.for_train = for_train

        self.texts = []
        self.labels = []
        for _, row in df.iterrows():
            self.texts.append(row['comment_text'])
            if for_train:
                self.labels.append([row[col] for col in LABELS])

    def __getitem__(self, idx):
        text = self.texts[idx]
        tokens = self.tokenizer.tokenize(text)
        tokens = tokens[:self.max_len-2]
        processed_tokens = ['[CLS]'] + tokens + ['[SEP]']

        input_ids = torch.tensor(self.tokenizer.convert_tokens_to_ids(processed_tokens))
        token_type_ids = torch.tensor([0] * len(processed_tokens))
        attention_mask = torch.tensor([1] * len(processed_tokens))

        outputs = (input_ids, token_type_ids, attention_mask)

        if self.for_train:
            label = self.labels[idx]
            label = torch.tensor(label)
            outputs += (label, )

        return outputs

    def __len__(self):
        return len(self.texts)

    def create_mini_batch(self, samples):
        outputs = list(zip(*samples))

        # zero pad 到同一序列長度
        input_ids = pad_sequence(outputs[0], batch_first=True)
        token_type_ids = pad_sequence(outputs[1], batch_first=True)
        attention_mask = pad_sequence(outputs[2], batch_first=True)

        batch_output = (input_ids, token_type_ids, attention_mask)
    
        if self.for_train:
            labels = torch.stack(outputs[3])
            batch_output += (labels, )

        return batch_output

In [9]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

dataset = MultiLabelDataset(tokenizer, df)

CUT_RATIO = 0.9
train_size = int(CUT_RATIO * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

In [23]:
batch_size = 2

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    collate_fn=dataset.create_mini_batch,
    shuffle=True
)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    collate_fn=dataset.create_mini_batch,
)

## Model

In [24]:
class BertForMultiLabelSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(
            input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())
            return logits, loss
        else:
            return logits

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

model = BertForMultiLabelSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels = len(LABELS)
)
model.to(device)

device: cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not 

BertForMultiLabelSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

## Train

In [33]:
def train_batch(model, data, optimizer, device):
    model.train()
    input_ids, token_type_ids, attention_mask, labels = [d.to(device) for d in data]

    _, loss = model(
        input_ids=input_ids,
        token_type_ids=token_type_ids,
        attention_mask=attention_mask,
        labels=labels
    )

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, valid_loader, device):
    model.eval()

    loss_avg = RunningAverage()
    all_preds = {l: [] for l in LABELS}
    all_labels = {l: [] for l in LABELS}

    with torch.no_grad():
        for data in tqdm(valid_loader, desc='evaluate'):
            input_ids, token_type_ids, attention_mask, labels = [d.to(device) for d in data]

            logits, loss = model(
                input_ids=input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss_avg.add(loss.item())

            preds = (torch.sigmoid(logits) > 0.5).int().cpu().numpy()
            labels = labels.cpu().numpy()
            for i, l in enumerate(LABELS):
                all_preds[l] += preds[:, i].tolist()
                all_labels[l] += labels[:, i].tolist()

    f1 = {
        l: metrics.f1_score(all_labels[l], all_preds[l])
        for l in LABELS
    }

    return loss_avg.get(), f1

In [34]:
lr = 0.00001
max_iter = 10000
show_per_iter = 200
valid_per_iter = 5000
save_per_iter = 5000
save_checkpoint_dir = 'models/'
model_prefix = 'en_toxic_label_'

assert save_per_iter % valid_per_iter == 0

optimizer = optim.Adam(model.parameters(), lr=lr)

i = 1
is_running = True
train_loss = RunningAverage()
model_paths = []
while is_running:
    for train_data in train_loader:
        loss = train_batch(model, train_data, optimizer, device)
        train_loss.add(loss)

        if i % show_per_iter == 0:
            print('train [{}]: loss={}'.format(i, train_loss.get()))
            train_loss.flush()

        if i % valid_per_iter == 0:
            loss, f1 = evaluate(model, valid_loader, device)
            print(f'valid: loss={loss}, f1={f1}')

        if i % save_per_iter == 0:
            path = os.path.join(save_checkpoint_dir, model_prefix + f'loss{loss:.5}/')
            print(f'save model at {path}')
            model.save_pretrained(path)
            model_paths.append(path)
        
        if i == max_iter:
            is_running = False
            break

        i += 1

train [200]: loss=0.28951802525669335
train [400]: loss=0.11745012611150742
train [600]: loss=0.11723310865461826
train [800]: loss=0.08834735627286136
train [1000]: loss=0.0935231857188046
train [1200]: loss=0.06744417215697468
train [1400]: loss=0.06993956945836544
train [1600]: loss=0.05925433183088899
train [1800]: loss=0.03853918759850785
train [2000]: loss=0.07057871089549735
train [2200]: loss=0.06184654379729181
train [2400]: loss=0.06368354315403849
train [2600]: loss=0.04747682274086401
train [2800]: loss=0.055768375953193755
train [3000]: loss=0.053984669451601804
train [3200]: loss=0.054268467320362106
train [3400]: loss=0.07124405334354378
train [3600]: loss=0.05656568333390169
train [3800]: loss=0.05101899838191457
train [4000]: loss=0.041767517544794824
train [4200]: loss=0.05705076205544174
train [4400]: loss=0.05174242081353441
train [4600]: loss=0.04896802713861689
train [4800]: loss=0.052057577148079875
train [5000]: loss=0.04646536611486227



valid: loss=0.055251650912019536, f1={'toxic': 0.8100032478077298, 'severe_toxic': 0.437619961612284, 'obscene': 0.7681307456588357, 'threat': 0.0, 'insult': 0.7161290322580645, 'identity_hate': 0.02469135802469136}
save model at models/en_toxic_label_loss0.055252/
train [5200]: loss=0.043382313501788304
train [5400]: loss=0.06510169170796871
train [5600]: loss=0.04811559410649352
train [5800]: loss=0.05491693730000406
train [6000]: loss=0.04151934144552797
train [6200]: loss=0.05607305980287492
train [6400]: loss=0.04717395420651883
train [6600]: loss=0.05378344219876453
train [6800]: loss=0.040816979463561436
train [7000]: loss=0.0469899137376342
train [7200]: loss=0.052113163790781986
train [7400]: loss=0.04955113299947698
train [7600]: loss=0.03846574262890499
train [7800]: loss=0.04221893641341012
train [8000]: loss=0.06273966826556716
train [8200]: loss=0.04813722028105985
train [8400]: loss=0.05002554221253377
train [8600]: loss=0.04502690893947147
train [8800]: loss=0.04060834


valid: loss=0.04475602889221653, f1={'toxic': 0.8170894526034712, 'severe_toxic': 0.450592885375494, 'obscene': 0.8145497912939772, 'threat': 0.0, 'insult': 0.7340025094102886, 'identity_hate': 0.0}
save model at models/en_toxic_label_loss0.044756/


## Predict

In [37]:
reload_checkpoint = model_paths[-1]

examples = [
    'Fuck you! You son of bitch',
    'I will kill you soon'
]
examples_df = pd.DataFrame(data={'comment_text': examples})

pred_dataset = MultiLabelDataset(tokenizer, examples_df, for_train=False)

pred_loader = DataLoader(
    dataset=pred_dataset,
    batch_size=batch_size,
    collate_fn=pred_dataset.create_mini_batch,
)

model = BertForMultiLabelSequenceClassification.from_pretrained(reload_checkpoint)
model.to(device)

pred_labels = []
with torch.no_grad():
    for data in tqdm(pred_loader, desc='predict'):
        input_ids, token_type_ids, attention_mask = [d.to(device) for d in data]

        logits = model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask
        )

        pred_labels += (torch.sigmoid(logits) > 0.5).int().cpu().tolist()

print('predict result: ', list(zip(examples, pred_labels)))


predict result:  [('Fuck you! You son of bitch', [1, 1, 1, 0, 1, 0]), ('I will kill you soon', [1, 0, 0, 0, 0, 0])]
